In [1]:
import pandas as pd
pd.options.display.float_format = '{:,.4f}'.format
pd.options.display.max_rows = 100
pd.options.display.max_columns = 40
import numpy as np
import os,random, math
from tqdm import tqdm
from copy import deepcopy
from collections import Counter

# Visualization
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import font_manager, rc
plt.rcParams['font.family'] = 'NanumGothic'
import platform
if platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
else:    
    rc('font', family='AppleGothic')

matplotlib.rcParams['axes.unicode_minus'] = False

# from dataprep.eda import plot, plot_correlation, plot_missing

# Warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error 

from catboost import CatBoostRegressor
from sklearn.model_selection import cross_val_score

In [2]:
import os
path = os.path.dirname(os.getcwd())
path = os.path.join(path, "ProcessedData")
train = pd.read_csv(os.path.join(path,"merged_train.csv"))
test = pd.read_csv(os.path.join(path,"merged_test.csv"))

In [5]:
train=train.drop_duplicates(keep='first')
test=test.drop_duplicates(keep='first')

In [26]:
tmp = train.groupby(['단지코드','임대건물구분']).count().iloc[:,:1].reset_index()
store = tmp[tmp['임대건물구분']=='상가'].reset_index(drop=True)[['단지코드','총세대수']].rename(columns={'총세대수':'상가수'})
apt = tmp[tmp['임대건물구분']=='아파트'].reset_index(drop=True)[['단지코드','총세대수']].rename(columns={'총세대수':'아파트수'})
total = pd.merge(apt,store,on='단지코드',how='left').fillna(0)
total['상가비율'] = total.apply(lambda x : x['상가수']/x['아파트수'],axis=1)
train = pd.merge(train,total[['단지코드','상가비율']],on='단지코드',how='left')

In [27]:
tmp = test.groupby(['단지코드','임대건물구분']).count().iloc[:,:1].reset_index()
store = tmp[tmp['임대건물구분']=='상가'].reset_index(drop=True)[['단지코드','총세대수']].rename(columns={'총세대수':'상가수'})
apt = tmp[tmp['임대건물구분']=='아파트'].reset_index(drop=True)[['단지코드','총세대수']].rename(columns={'총세대수':'아파트수'})
total = pd.merge(apt,store,on='단지코드',how='left').fillna(0)
total['상가비율'] = total.apply(lambda x : x['상가수']/x['아파트수'],axis=1)
test = pd.merge(test,total[['단지코드','상가비율']],on='단지코드',how='left')

In [28]:
train.loc[50:100,'상가비율']

50    0.0000
51    0.0000
52    0.0000
53    0.0000
54    0.0000
55    0.0000
56    0.0000
57    0.0000
58    0.0000
59    0.0000
60    0.0000
61    0.0000
62    0.0000
63    0.0000
64    0.0000
65    0.0000
66    0.0000
67    0.0000
68    0.0000
69    0.0000
70    0.0000
71    1.0000
72    1.0000
73    0.0000
74    0.0000
75    0.0000
76    0.0000
77    0.0000
78    0.0000
79    4.3333
80    4.3333
81    4.3333
82    0.0000
83    0.0000
84    0.0000
85    4.0000
86    4.0000
87    0.0000
88    0.0000
89    0.0000
90    0.0000
91    0.0000
92    0.0000
93    0.0000
94    0.0000
95    0.0000
96    0.0000
97    0.0000
98    0.0000
99    0.5000
100   0.5000
Name: 상가비율, dtype: float64